In [1]:
%load_ext dotenv
%dotenv
import os
from neo4j import GraphDatabase


AUTH = (os.getenv("NEO4J_USER"), os.getenv("NEO4J_PASSWORD"))

with GraphDatabase.driver(os.getenv("NEO4J_URL"), auth=(AUTH)) as driver:
    driver.verify_connectivity()

In [2]:
from importlib import reload
import openai_assistant

reload(openai_assistant)

<module 'openai_assistant' from '/Users/oskarhane/Development/tmp/neo4j-genai-design/openai_assistant.py'>

In [61]:
from custom_retrievers import movie
import retriever_router
import neo4j_retrievers
import neo4j_rag
reload(neo4j_retrievers)
reload(movie)
reload(retriever_router)
reload(neo4j_rag)

llm = openai_assistant.OpenAIAssitant()

movie_plot_similarity = neo4j_retrievers.VectorRetriever(
    driver=driver,
    llm=llm, 
    name="movie_plot_similarity",
    index_name="moviePlotsEmbedding"
)
text2cypher_retriever = neo4j_retrievers.Text2Cypher(driver=driver, llm=llm)
movie_retriever = movie.MovieRetriever(driver=driver)

agentic_retriever = retriever_router.RetrieverRouter(
    llm=llm, 
    retrievers=[movie_plot_similarity, movie_retriever, text2cypher_retriever]
)

rag_app = neo4j_rag.Neo4jRAG(llm=llm, retriever=agentic_retriever)
# or
single_retriever_rag_app = neo4j_rag.Neo4jRAG(llm=llm, retriever=text2cypher_retriever)


In [64]:
print("------ Agentic retriever ------")
response = rag_app.handle_input('What year was the movie "Casino" released?')
print("--- output ---")
print(response)

print("------ Single retriever ------")
response = single_retriever_rag_app.handle_input('What year was the movie "Casino" released?')
print("--- output ---")
print(response)

------ Agentic retriever ------
Expanded questions: ["What year was the movie 'Casino' released?"]
Updated question: What year was the movie 'Casino' released?
MovieRetriever invoked: 'Casino' 'exact'
Response from tools: ['[{"movie": {"title": "Casino", "year": 1995, "plot": "Greed, deception, money, power, and murder occur between two mobster best friends and a trophy wife over a gambling empire."}, "actors": [{"name": "Robert De Niro", "role": "Sam \'Ace\' Rothstein"}, {"name": "Joe Pesci", "role": "Nicky Santoro"}, {"name": "Sharon Stone", "role": "Ginger McKenna"}, {"name": "James Woods", "role": "Lester Diamond"}]}]']
Unanswered questions: []
--- output ---
The movie "Casino" was released in 1995.
------ Single retriever ------
Text2Cypher invoked: What year was the movie "Casino" released?
MATCH (m:Movie {title: "Casino"}) RETURN m.released AS released
--- output ---
The movie "Casino" was released in 1995.


In [71]:
response = rag_app.handle_input('What movie is about two mobster best friends, and what year was it released? And who were some actors playing roles in it?')
print("--- output ---")
print(response)


Expanded questions: ['What movie is about two mobster best friends?', 'What year was the movie released?', 'Who were some actors playing roles in the movie?']
Updated question: What movie is about two mobster best friends and a trophy wife over a gambling empire?
VectorRetriever invoked: two mobster best friends and a trophy wife over a gambling empire
Response from tools: ['{"movie": {"title": "Casino", "year": 1995, "plot": "Greed, deception, money, power, and murder occur between two mobster best friends and a trophy wife over a gambling empire."}}']
Updated question: What year was the movie 'Casino' released?
MovieRetriever invoked: 'Casino' 'exact'
Response from tools: ['[{"movie": {"title": "Casino", "year": 1995, "plot": "Greed, deception, money, power, and murder occur between two mobster best friends and a trophy wife over a gambling empire."}, "actors": [{"name": "Robert De Niro", "role": "Sam \'Ace\' Rothstein"}, {"name": "Joe Pesci", "role": "Nicky Santoro"}, {"name": "Shar

In [46]:
response = rag_app.handle_input('How many actors were in the movie "Casino"')
print("--- output ---")
print(response)

Expanded questions: ["How many actors were in the movie 'Casino'?"]
Updated question: How many actors played roles in the movie 'Casino'?
Text2Cypher invoked: How many actors played roles in the movie 'Casino'?
MATCH (a:Actor)-[r:ACTED_IN]->(m:Movie {title:'Casino'})
RETURN COUNT(DISTINCT a) AS actorCount;
Response from tools: ['{"query": "MATCH (a:Actor)-[r:ACTED_IN]->(m:Movie {title:\'Casino\'})\\nRETURN COUNT(DISTINCT a) AS actorCount;", "records": [{"actorCount": 4}]}']
Unanswered questions: []
--- output ---
According to the provided information, there were 4 actors in the movie "Casino."


In [73]:
response = rag_app.handle_input('What are two the highest rated movies Robert De Niro has acted in? What are the plots in those movies?')
print("--- output ---")
print(response)

Expanded questions: ['What are two of the highest rated movies that Robert De Niro has acted in?', 'What is the plot of the first movie?', 'What is the plot of the second movie?']
Updated question: What are two the highest rated movies Robert De Niro has acted in?
Text2Cypher invoked: Highest rated movies Robert De Niro has acted in
MATCH (p:Person)-[a:ACTED_IN]->(m:Movie) WHERE p.name = "Robert De Niro" RETURN m.title ORDER BY m.imdbRating DESC LIMIT 1
Response from tools: ['{"query": "MATCH (p:Person)-[a:ACTED_IN]->(m:Movie) WHERE p.name = \\"Robert De Niro\\" RETURN m.title ORDER BY m.imdbRating DESC LIMIT 1", "records": [{"m.title": "Godfather: Part II, The"}]}']
Updated question: What is the plot of the first movie in a list of movies with their plots?
Text2Cypher invoked: What is the plot of the first movie in a list of movies with their plots?
MATCH (m:Movie) RETURN m.plot ORDER BY m.released LIMIT 1

Response from tools: ['{"query": "MATCH (m:Movie) RETURN m.plot ORDER BY m.rel